In [2]:
!pip install faiss-cpu
!pip install pandas numpy


In [3]:
import shutil

# Copy the required files from the dataset folder to the current directory
shutil.copy("/kaggle/input/my-dataset/math_index.faiss", "math_index.faiss")
shutil.copy("/kaggle/input/my-dataset/answers.json", "answers.json")


'answers.json'

In [4]:
import pandas as pd

# Load training data
train_df = pd.read_csv('/kaggle/input/my-dataset/train.csv')
train_df.head()


,problem,level,type,solution,answer
0,The United States Postal Service charges an ex...,Level 3,Prealgebra,We calculate the desired ratio for each envelo...,3
1,How many integers between 1000 and 2000 have a...,Level 4,Prealgebra,"A number with 15, 20 and 25 as factors must be...",3
2,"Given that $n$ is an integer and $0 < 4n <30$,...",Level 2,Prealgebra,"Dividing by $4$, we have $0<n<7\frac{1}{2}$. T...",28
3,How many integers between $100$ and $150$ have...,Level 4,Prealgebra,We will break up the problem into cases based ...,18
4,Regular pentagon $ABCDE$ and regular hexagon $...,Level 4,Prealgebra,We know that the sum of the degree measures of...,132


In [5]:
!pip install -q faiss-cpu sentence-transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.8 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 88.5 MB/s eta 0:00:00:00:0100:01


In [6]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import json


2025-07-03 05:25:38.656610: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751520338.855517      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751520338.910175      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [7]:
# Step 3
questions = train_df['problem'].tolist()
answers = train_df['answer'].tolist()

model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(questions, show_progress_bar=True)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/235 [00:00<?, ?it/s]

In [8]:
import faiss
import numpy as np

embeddings = np.array(embeddings).astype('float32')
index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)

faiss.write_index(index, "math_index.faiss")
print("✅ FAISS index saved as math_index.faiss")


✅ FAISS index saved as math_index.faiss


In [9]:
import json

index_to_answer = {i: ans for i, ans in enumerate(answers)}

with open("answers.json", "w") as f:
    json.dump(index_to_answer, f)

print(f"✅ Saved {len(index_to_answer)} answers to answers.json")


✅ Saved 7498 answers to answers.json


In [10]:
from sentence_transformers import SentenceTransformer
import faiss
import json
import numpy as np

class MathChatbot:
    def __init__(self, index_path="math_index.faiss", answer_path="answers.json"):
        self.retriever = SentenceTransformer("all-MiniLM-L6-v2", device="cpu")
        self.index = faiss.read_index(index_path)
        with open(answer_path, "r") as f:
            self.index_to_answer = json.load(f)

    def get_response(self, query):
        embedding = self.retriever.encode([query])[0].astype('float32')
        D, I = self.index.search(np.array([embedding]), k=1)
        return self.index_to_answer[str(I[0][0])]
        




In [11]:
# ✅ First, ensure bot is ready
bot = MathChatbot(index_path="math_index.faiss", answer_path="answers.json")
print("🤖 MathBot is ready! Ask your math questions (type 'exit' to quit)\n")

# ✅ Simple chat loop
while True:
    query = input("🧑 You: ")
    if query.lower() in ['exit', 'quit']:
        print("👋 Goodbye!")
        break
    
    response = bot.get_response(query)
    print("🤖 MathBot:", response, "\n")


🤖 MathBot is ready! Ask your math questions (type 'exit' to quit)



KeyboardInterrupt: Interrupted by user

In [12]:
!pip install gradio

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.3/54.3 MB 33.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.9/323.9 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.5/95.5 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 108.5 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 4.1 MB/s eta 0:00:00


In [13]:
import time
import gradio as gr

# Your existing chatbot
bot = MathChatbot(index_path="math_index.faiss", answer_path="answers.json")

def chat_fn(message, chat_history):
    time.sleep(0.3)  # ensure loader shows
    chat_history = chat_history or []
    chat_history.append({"role": "user", "content": message})
    response = bot.get_response(message)
    chat_history.append({"role": "assistant", "content": response})
    return chat_history, ""

with gr.Blocks(css="""
  body {
    margin: 0; padding: 0;
    background: #1e1e1e;
    font-family: 'Segoe UI', sans-serif;
  }
  #title {
    text-align: center;
    color: #e5e5e5;
    margin-top: 1rem;
    margin-bottom: 0.5rem;
  }
  #chatbot {
    height: 500px;
    overflow-y: auto;
    border-radius: 0;              /* remove rounded border */
    padding: 1rem;
    background: #2a2a2a;           /* deep gray background */
  }
  .gr-chatbot .message.user .bubble {
    background: #3b3b3b !important; /* darker gray user bubble */
    color: #e5e5e5 !important;
  }
  .gr-chatbot .message.bot .bubble {
    background: #333333 !important;  /* slightly darker bot bubble */
    color: #cfcfcf !important;
  }
  #input_row {
    margin-top: auto !important;
    background: #2a2a2a;           /* match chat background */
    padding: 0.75rem 1rem;
  }
  #textbox textarea {
    resize: none !important;
    background: #3b3b3b !important; /* input box gray */
    color: #e5e5e5 !important;
    border: none !important;        /* remove border */
    border-radius: 8px !important;
    padding: 0.75rem !important;
  }
  .gr-button {
    background: #2563eb !important; /* keep send button blue */
    color: white !important;
    border-radius: 8px !important;
    border: none !important;        /* remove border */
    box-shadow: none !important;
  }
  .gr-button:hover {
    background: #1e40af !important;
  }
  .gr-button:focus {
    box-shadow: none !important;
  }
  .gr-button-secondary {
    background: none !important;    /* clear chat button transparent */
    color: #888 !important;
  }
  .gr-button-secondary:hover {
    color: #bbb !important;
  }
""") as demo:
    gr.Markdown("## 🤖 MathGPT", elem_id="title")

    # Use the new 'messages' style
    chatbot = gr.Chatbot(elem_id="chatbot", type="messages")
    msg     = gr.Textbox(placeholder="Type your math question...", show_label=False, elem_id="textbox", lines=1)
    send_btn= gr.Button("➤ Send", variant="primary")
    clear_btn = gr.Button("🧹 Clear Chat", variant="secondary")

    msg.submit(chat_fn, [msg, chatbot], [chatbot, msg])
    send_btn.click(chat_fn, [msg, chatbot], [chatbot, msg])
    clear_btn.click(lambda: ([], ""), None, [chatbot, msg])

demo.launch()

* Running on local URL:  http://127.0.0.1:7860
It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

* Running on public URL: https://2f2296b225e34c7484.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
